In [1]:
import pandas as pd
import networkx as nx

path = '/Users/paulocarmo/repositories/nyt-lp/'

In [2]:
df = pd.read_pickle('{}/data_v0.pkl'.format(path))
df

,_id,pub_date,chosen_type,text,what,when,where,who,why,how,coords
0,nyt://article/462daa07-2b8b-5833-afb5-209a968a...,2015-07-01 00:03:44,lead_paragraph,LONDON — Rossini’s opera “Guillaume Tell” has ...,is forced to shoot an arrow,None,LONDON,the hero,Rossini ’s opera “ Guillaume Tell ”,has long been known for the scene in which the,"(51.5073219, -0.1276474)"
1,nyt://article/0a038481-b145-57b3-a94b-701f0e9b...,2015-07-01 00:05:47,lead_paragraph,PARIS — Sometimes a metaphor lands at your fee...,None,last Friday,PARIS,None,None,"— Sometimes a metaphor lands at your feet , li...","(48.8588897, 2.3200410217200766)"
2,nyt://slideshow/3f39a127-f709-5e2e-8c94-2864fe...,2015-07-01 00:07:39,abstract,"Images of calcio storico in Florence, Italy, a...",None,None,"Florence , Italy",None,None,a centuries-old competition with few rules and...,"(43.7698712, 11.2555757)"
3,nyt://article/1565acdd-a256-5f81-93df-232b1acb...,2015-07-01 00:09:33,lead_paragraph,"Theodore Weesner, a novelist who mined his way...",was 79,Thursday,"Portsmouth , N.H",novelist,None,his wayward youth for the stuff of his celebra...,"(43.0702223, -70.7548621)"
4,nyt://article/694f7d7c-304a-5370-aff0-731aa012...,2015-07-01 00:19:40,lead_paragraph,"DEIR AL-BALAH, Gaza Strip — One bomb hit a Ham...",exploded in a trash can,A few days later,Gaza Strip,another,a small one,in northern Gaza .,"(31.4432234, 34.360007)"
...,...,...,...,...,...,...,...,...,...,...,...
157745,nyt://article/51f01df3-5548-5f97-8d94-ea64d449...,2021-07-31 19:28:13,lead_paragraph,"PROVINCETOWN, Mass. — By the Fourth of July, P...",None,the Fourth of July,"PROVINCETOWN , Mass.",None,Restaurants,a prepandemic thrum .,"(42.0525612, -70.1858639)"
157746,nyt://video/9afc7520-16a5-5cf8-8d37-e4d4f7d9fd05,2021-07-31 19:34:53,abstract,"Dr. Tedros Adhanom Ghebreyesus, the World Heal...",None,None,Africa,None,"Dr. Tedros Adhanom Ghebreyesus , the World Hea...",a worldwide rise in Covid-19 cases has dispara...,"(11.5024338, 17.7578122)"
157747,nyt://article/5300b238-e7cc-5d4e-bda0-59c450b1...,2021-07-31 21:13:08,lead_paragraph,At least 233 staff members at two major San Fr...,"vaccinated , tested positive for the coronavirus",this month,San Francisco,At least 233 staff members at two major San Fr...,None,At least 233 staff members at two major San Fr...,"(37.7790262, -122.419906)"
157748,nyt://article/8616f3bd-e981-5c1b-8532-146f126b...,2021-07-31 21:34:38,lead_paragraph,"PARIS — Pascale Collino, 64, is far more afrai...",None,None,PARIS,None,None,is far more afraid of the Covid-19 vaccines th...,"(48.8588897, 2.3200410217200766)"


In [8]:
from tqdm import tqdm

def make_networkx(df,
                    id_feature='_id', list_features=[
                        'pub_date', 'what', 'when', 'where', 'who', 'why', 'how', 'coords'
                    ],
):
    def create_edge(value1, value2, node_type1, node_type2):
        if not(value1 is None) and not(value2 is None):
            G.add_edge(value1, value2, edge_type=f'{node_type1}_{node_type2}')
            G.nodes[value1]['node_type'] = node_type1
            G.nodes[value2]['node_type'] = node_type2

    G = nx.Graph()
    for _, row in tqdm(df.iterrows()):
        for feature in list_features:
            create_edge(row[id_feature], row[feature], id_feature, feature)
            G.nodes[row[id_feature]]['chosen_type'] = row['chosen_type']
            G.nodes[row[id_feature]]['text'] = row['text']
    return G

G = make_networkx(df)

157750it [00:37, 4185.67it/s]


In [9]:
[G.nodes[node]['text'] for node in G.nodes() if G.nodes[node]['node_type'] == '_id']

['LONDON — Rossini’s opera “Guillaume Tell” has long been known for the scene in which the hero is forced to shoot an arrow at an apple perched on the head of his son. Now, however, at least in London, it may be memorable for a different reason: A sequence in the Royal Opera House production here includes full frontal nudity and suggests a gang rape.',
 'PARIS — Sometimes a metaphor lands at your feet, literally. This happened last Friday, midway through Riccardo Tisci’s men’s wear show for Givenchy in Paris, when a weighty jailer’s key embellishing a necklace worn by a model slipped off and went flying, thudding into dust in front of an audience member.',
 'Images of calcio storico in Florence, Italy, a centuries-old competition with few rules and the sort of human wreckage generally associated with the gladiators.',
 'Theodore Weesner, a novelist who mined his wayward youth for the stuff of his celebrated first novel, “The Car Thief,” and whose half-dozen other books earned plaudits 